In [65]:
import numpy as np
import pandas as pd
import os
import re
import shutil

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator

import cv2
from PIL import Image
matplotlib.use('Agg')  
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [66]:
# Define dataset directory
images_dir = "images"

# Training parameters
img_size = (10, 10)  # Resize images to 64x64
batch_size = 32
epochs = 10

# Dictionary to store max accuracies
max_accuracies = {}

In [67]:
# Function to build a simple CNN model
def cnn_model(input_shape):
	model = Sequential([
		Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
		MaxPooling2D(2,2),
		Conv2D(64, (3,3), activation='relu'),
		MaxPooling2D(2,2),
		Flatten(),
		Dense(128, activation='relu'),
		Dense(5, activation='softmax')  # Adjust output neurons based on classes
	])
	model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [68]:
# Iterate through resolution folders
for resolution_folder in sorted(os.listdir(images_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	# sort resolution folders to ensure consistent order
	resolution_path = os.path.join(images_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		# Iterate through method folders inside resolution
		for method_folder in os.listdir(resolution_path):
			method_path = os.path.join(resolution_path, method_folder)

			if os.path.isdir(method_path):
				print(f"\nTraining on {resolution_folder}/{method_folder}...")

				# Data Preprocessing
				datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
				train_data = datagen.flow_from_directory(
					method_path, target_size=img_size, batch_size=batch_size,
					class_mode='categorical', subset='training'
				)
				val_data = datagen.flow_from_directory(
					method_path, target_size=img_size, batch_size=batch_size,
					class_mode='categorical', subset='validation'
				)

				# Build and train CNN model
				model = cnn_model(input_shape=(img_size[0], img_size[1], 3))
				history = model.fit(train_data, validation_data=val_data, epochs=epochs, verbose=1)

				# Get max validation accuracy
				max_acc = max(history.history['val_accuracy'])
				key = f"{resolution_folder} - {method_folder}"
				max_accuracies[key] = max_acc
				print(f"Max accuracy for {key}: {max_acc:.4f}")

				# Save model
				model_dir = os.path.join("models", resolution_folder, method_folder)
				os.makedirs(model_dir, exist_ok=True)
				model_name = f"{resolution_folder}_{method_folder}.h5"
				model.save(os.path.join(model_dir, model_name))
				print(f"Model saved to {model_dir}")


Training on 4px/Chargaff-Composante-Diversite...
Found 5419 images belonging to 5 classes.
Found 1352 images belonging to 5 classes.
Epoch 1/10
170/170 [==============================] - 4s 21ms/step - loss: 0.9872 - accuracy: 0.6510 - val_loss: 0.9577 - val_accuracy: 0.6494
Epoch 2/10
170/170 [==============================] - 2s 14ms/step - loss: 0.9175 - accuracy: 0.6617 - val_loss: 0.9538 - val_accuracy: 0.6442
Epoch 3/10
170/170 [==============================] - 2s 14ms/step - loss: 0.9021 - accuracy: 0.6704 - val_loss: 0.9350 - val_accuracy: 0.6450
Epoch 4/10
170/170 [==============================] - 2s 13ms/step - loss: 0.8796 - accuracy: 0.6815 - val_loss: 0.9300 - val_accuracy: 0.6479
Epoch 5/10
170/170 [==============================] - 2s 13ms/step - loss: 0.8671 - accuracy: 0.6894 - val_loss: 0.9135 - val_accuracy: 0.6464
Epoch 6/10
170/170 [==============================] - 2s 13ms/step - loss: 0.8635 - accuracy: 0.6898 - val_loss: 0.9230 - val_accuracy: 0.6612
Epoch 7/

c:\Users\theof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Found 5419 images belonging to 5 classes.
Found 1352 images belonging to 5 classes.
Epoch 1/10
170/170 [==============================] - 4s 18ms/step - loss: 1.0032 - accuracy: 0.6422 - val_loss: 0.9775 - val_accuracy: 0.6516
Epoch 2/10
170/170 [==============================] - 2s 12ms/step - loss: 0.9795 - accuracy: 0.6510 - val_loss: 0.9677 - val_accuracy: 0.6516
Epoch 3/10
170/170 [==============================] - 2s 12ms/step - loss: 0.9707 - accuracy: 0.6510 - val_loss: 0.9752 - val_accuracy: 0.6516
Epoch 4/10
170/170 [==============================] - 2s 11ms/step - loss: 0.9555 - accuracy: 0.6534 - val_loss: 0.9371 - val_accuracy: 0.6516
Epoch 5/10
170/170 [==============================] - 2s 12ms/step - loss: 0.9381 - accuracy: 0.6634 - val_loss: 0.9046 - val_accuracy: 0.6679
Epoch 6/10
170/170 [==============================] - 2s 12ms/step - loss: 0.9188 - accuracy: 0.6784 - val_loss: 0.8807 - val_accuracy: 0.7389
Epoch 7/10
170/170 [==============================] - 2s 1

In [69]:
# Create positions for bars
num_bars = len(max_accuracies)
x_positions = np.arange(num_bars)

# Add extra space every 10 bars
for i in range(10, num_bars, 10):  
	x_positions[i:] += 1  # Shift everything after every 10th bar

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Max Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\3164836006.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [70]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))

# Organize data for plotting
data = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods}

# Define a color map for better distinction
colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors

# Plot the lines
plt.figure(figsize=(16, 10))
for i, (method, accuracies) in enumerate(data.items()):
	plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, label=method, color=colors(i))

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution and Method", fontsize=14)

# Move legend outside the plot for better clarity
plt.legend(title="Method", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)

# Add a grid with transparency
plt.grid(True, linestyle="--", alpha=0.6)

# Improve layout to fit legend properly
plt.tight_layout()

# Show plot
plt.show()

plt.savefig("../../imgs/graphs/cnn_validation_accuracy.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\2751164684.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_27196\2751164684.py:31: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [71]:
# Extract and sort resolutions numerically
resolutions = sorted(set(k.split(" - ")[0] for k in max_accuracies.keys()), key=lambda x: int(x.replace("px", "")))

# Extract unique methods and split into two groups of 5
methods = sorted(set(k.split(" - ")[1] for k in max_accuracies.keys()))
methods_group1 = methods[:5]
methods_group2 = methods[5:]

# Organize data for plotting
data1 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group1}
data2 = {method: [max_accuracies.get(f"{res} - {method}", None) for res in resolutions] for method in methods_group2}

# Create subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 10), sharex=True, sharey=True)

colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors

# Plot first group
for method, accuracies in data1.items():
	axs[0].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group1.index(method)))
axs[0].set_title("CNN Validation Accuracy (Group 1)")
axs[0].set_ylabel("Max Accuracy")
axs[0].legend(title="Method")
axs[0].grid(True)

# Plot second group
for method, accuracies in data2.items():
	axs[1].plot(resolutions, accuracies, marker="o", label=method, color=colors(methods_group2.index(method)))
axs[1].set_title("CNN Validation Accuracy (Group 2)")
axs[1].set_xlabel("Resolution")
axs[1].set_ylabel("Max Accuracy")
axs[1].legend(title="Method")
axs[1].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/cnn_validation_accuracy_groups.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\4277257368.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap("tab10", len(methods_group1))  # Use "tab10" color map with enough colors
C:\Users\theof\AppData\Local\Temp\ipykernel_27196\4277257368.py:37: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [72]:
# confusion matrix
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from keras.models import load_model

# Load the model
model_path = "models/100px/Chargaff-Composante-Diversite/100px_Chargaff-Composante-Diversite.h5"
model = load_model(model_path)
model.summary()

# Load the test data
test_data_dir = 'images/100px/Chargaff-Composante-Diversite'

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
	test_data_dir,
	target_size=(img_size[0], img_size[1]),
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)
# Get the true labels
true_labels = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Get the predicted labels
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_labels = np.argmax(predictions, axis=1)
# Generate the confusion matrix
cm = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix')
plt.show()

plt.savefig("../../imgs/graphs/cnn_confusion_matrix_100px.png")

Model: "sequential_96"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_192 (Conv2D)         (None, 8, 8, 32)          896       
                                                                 
 max_pooling2d_192 (MaxPool  (None, 4, 4, 32)          0         
 ing2D)                                                          
                                                                 
 conv2d_193 (Conv2D)         (None, 2, 2, 64)          18496     
                                                                 
 max_pooling2d_193 (MaxPool  (None, 1, 1, 64)          0         
 ing2D)                                                          
                                                                 
 flatten_96 (Flatten)        (None, 64)                0         
                                                                 
 dense_192 (Dense)           (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\4260248153.py:38: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [73]:
# Define dataset directory
mosaics_dir = "C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/res"

# Training parameters
img_size = (20, 50)  # Resize images to 64x64
batch_size = 32
epochs = 10

# Dictionary to store max accuracies
max_accuracies_mosaics = {}

In [74]:
# mosaic version
# Iterate through resolution folders
for resolution_folder in sorted(os.listdir(images_dir), key=lambda x: int(re.search(r'\d+', x).group())):
	resolution_path = os.path.join(mosaics_dir, resolution_folder)

	if os.path.isdir(resolution_path):
		print(f"\nTraining on {resolution_folder}...")

		# Data Preprocessing
		datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
		train_data = datagen.flow_from_directory(
			resolution_path, target_size=img_size, batch_size=batch_size,
			class_mode='categorical', subset='training'
		)
		val_data = datagen.flow_from_directory(
			resolution_path, target_size=img_size, batch_size=batch_size,
			class_mode='categorical', subset='validation'
		)

		# Build and train CNN model
		model = cnn_model(input_shape=(img_size[0], img_size[1], 3))
		history = model.fit(train_data, validation_data=val_data, epochs=epochs, verbose=1)

		# Get max validation accuracy
		max_acc_mos = max(history.history['val_accuracy'])
		max_accuracies_mosaics[resolution_folder] = max_acc_mos
		print(f"Max accuracy for {resolution_folder}: {max_acc_mos:.4f}")

		# Save model
		# model_dir = os.path.join("models/mosaics", resolution_folder)
		# os.makedirs(model_dir, exist_ok=True)
		# model_name = f"{resolution_folder}_mosaic.h5"
		# model.save(os.path.join(model_dir, model_name))
		# print(f"Model saved to {model_dir}")


Training on 4px...
Found 5419 images belonging to 5 classes.
Found 1352 images belonging to 5 classes.
Epoch 1/10
170/170 [==============================] - 5s 23ms/step - loss: 0.9636 - accuracy: 0.6507 - val_loss: 0.9474 - val_accuracy: 0.6376
Epoch 2/10
167/170 [============================>.] - ETA: 0s - loss: 0.9011 - accuracy: 0.6788

KeyboardInterrupt: 

In [61]:
# Create positions for bars
num_bars = len(max_accuracies_mosaics)
x_positions = np.arange(num_bars)

# Add extra space every 10 bars
for i in range(10, num_bars, 10):  
	x_positions[i:] += 1  # Shift everything after every 10th bar

plt.figure(figsize=(15, 10))
plt.bar(x_positions, max_accuracies_mosaics.values(), color='blue', width=0.5)
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Custom x-ticks for readability
plt.xticks(x_positions, max_accuracies_mosaics.keys(), rotation=90)

plt.yticks(np.arange(0, 1.1, 0.2))

plt.ylabel("Max Accuracy")
plt.title("CNN Validation Results")
plt.tight_layout()
plt.show()

plt.savefig("../../imgs/graphs/cnn_validation_accuracy_mosaics.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\2020362206.py:21: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [62]:
# Extract and sort resolutions numerically
resolutions = sorted(max_accuracies_mosaics.keys(), key=lambda x: int(x.replace("px", "")))

# Get max accuracy values in the correct order
accuracies = [max_accuracies_mosaics[res] for res in resolutions]

# Plot the accuracy per resolution
plt.figure(figsize=(12, 6))
plt.plot(resolutions, accuracies, marker="o", linestyle="-", linewidth=2, markersize=8, color="blue", label="Accuracy")

# Labels and title
plt.xlabel("Resolution", fontsize=12)
plt.ylabel("Max Accuracy", fontsize=12)
plt.title("CNN Validation Accuracy by Resolution", fontsize=14)

# Add data points on the plot
for i, acc in enumerate(accuracies):
    plt.text(resolutions[i], acc, f"{acc:.2f}", fontsize=10, ha="right")

# Grid and legend
plt.grid(True, linestyle="--", alpha=0.6)
plt.legend(loc="lower right")

# Save and show plot
plt.tight_layout()
plt.show()
plt.savefig("../../imgs/graphs/cnn_validation_accuracy_mosaics_line.png")

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\331704733.py:26: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [63]:
# Load the model for the new version
new_model_path = "models/mosaics/100px/100px_mosaic.h5"
new_model = load_model(new_model_path)
new_model.summary()

# Load the test data for the new version
new_test_data_dir = 'C:/Users/theof/OneDrive/Documents/Github/genome_color_unpickler/res/100px'

new_test_datagen = ImageDataGenerator(rescale=1./255)
new_test_generator = new_test_datagen.flow_from_directory(
	new_test_data_dir,
	target_size=(img_size[0], img_size[1]),
	batch_size=batch_size,
	class_mode='categorical',
	shuffle=False
)

# Get the true labels for the new version
new_true_labels = new_test_generator.classes
new_class_labels = list(new_test_generator.class_indices.keys())

# Get the predicted labels for the new version
new_predictions = new_model.predict(new_test_generator, steps=len(new_test_generator), verbose=1)
new_predicted_labels = np.argmax(new_predictions, axis=1)

# Generate the confusion matrix for the new version
new_cm = confusion_matrix(new_true_labels, new_predicted_labels)

# Plot the confusion matrix for the new version
plt.figure(figsize=(10, 8))
sns.heatmap(new_cm, annot=True, fmt='d', cmap='Blues', xticklabels=new_class_labels, yticklabels=new_class_labels)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title('Confusion Matrix for New Version')
plt.show()

plt.savefig("../../imgs/graphs/cnn_confusion_matrix_mosaics_100px.png")

Model: "sequential_55"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_110 (Conv2D)         (None, 18, 48, 32)        896       
                                                                 
 max_pooling2d_110 (MaxPool  (None, 9, 24, 32)         0         
 ing2D)                                                          
                                                                 
 conv2d_111 (Conv2D)         (None, 7, 22, 64)         18496     
                                                                 
 max_pooling2d_111 (MaxPool  (None, 3, 11, 64)         0         
 ing2D)                                                          
                                                                 
 flatten_55 (Flatten)        (None, 2112)              0         
                                                                 
 dense_110 (Dense)           (None, 128)             

C:\Users\theof\AppData\Local\Temp\ipykernel_27196\3084799137.py:35: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
